In [ ]:
import qrcode
import base64
from io import BytesIO
from PIL import Image
import pandas as pd
import numpy as np
import copy 
import PIL
import tkinter
from qrcode.image.styledpil import StyledPilImage
from qrcode.image.styles.colormasks import RadialGradiantColorMask
import copy
from pathlib import Path
from time import time

In [ ]:
def generateQRSRC(sku):
    qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=3,
    border=0,
    )
    qr.add_data(sku)
    qr.make(fit=True)

    img = qr.make_image(fill_color="black", back_color="white") #of (image_factory=StyledPilImage, color_mask=RadialGradiantColorMask())
    im_file = BytesIO()
    img.save(im_file, format="JPEG")
    im_bytes = im_file.getvalue()
    im_b64 = base64.b64encode(im_bytes)

    img_source = f'data:image/jpeg;base64,{im_b64.decode("utf-8")}'
    return img_source

In [ ]:
generateQRSRC('asdf')

In [ ]:
df = pd.read_csv('./Vaulted Inventory & Consignment Management Verson.csv')
df.columns = [c.lower() for c in df.columns]
df['vintage'] = df['vintage'].astype(int).astype(str)

In [ ]:
df['url'] = df['sku']
df['tokenId'] = df['sku'].apply(lambda x: x.split('/')[-1])
df['sku'] = df['sku'].apply(lambda x: x.split('/')[-2])


In [ ]:
df['url'].values

In [ ]:
with open('./template.html', 'r') as f:
    lines = f.readlines()
html = ''.join(lines)

In [ ]:
template = """<h4>COMPANY_NAME (SKU_CODE/TOKEN_ID)</h4>
<img src="SOURCE_BASE64" style="float:right">
<h5>VAULT_NAME</h5>
<h5>PRODUCER</h5>
<h5>RELEASE, VINTAGE</h5>

<p style="float:right">Powered by <img src="https://www.google.com/s2/favicons?domain=fanfire.ai" style=”width:40px;height:20px;"> Fanfire</p>"""

In [ ]:
items = []
for i, row in df.iterrows():
    item = copy\
    .deepcopy(template)\
    .replace("COMPANY_NAME", row['company'])\
    .replace("VAULT_NAME", row['vault'])\
    .replace("PRODUCER", row['producer'])\
    .replace("RELEASE", row['release'])\
    .replace("VINTAGE", row['vintage'])\
    .replace("SKU_CODE", row['sku'])\
    .replace("TOKEN_ID", row['tokenId'])\
    .replace("SOURCE_BASE64", generateQRSRC(row['sku']))
    
    items.append(item)

In [ ]:
run_dir = Path(str(time()))
run_dir.mkdir()
page_nr = 0 
page = copy.deepcopy(html)
for item_nr, item in enumerate(items):
    print(item_nr, page_nr) 
    page = page.replace(f'ITEM_{(item_nr%10)+1}_HTML', item)
    if (item_nr + 1) % 10 == 0:
        with open(run_dir/f'page-{page_nr}.html', 'w') as f:
            f.write(page)
        page_nr = page_nr + 1
        page = copy.deepcopy(html)
with open(run_dir/f'page-{page_nr}.html', 'w') as f:
    f.write(page)